# Volume 3: Pandas 1 (Introduction)
    Nathan Kibanoff
    BUDS Training Program
    9 August 2019

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

## Problem 1
Read in the file *crime_data.csv* into a pandas object. The file contains data on types of crimes in the U.S. from 1960 to 2016. Set the index as the column *Year*.

Create a new column 'Rate' which has the crime rate for each year. Use this table to answer the following questions:

1) What is the mean crime rate? In what years was the crime rate above the mean?

2) On average, using the mean, what is the least common crime?

3) Identify two crimes which have close means. Do these two crimes have any kind of correlation from year to year? If so, what is this correlation?

In [3]:
crimes=pd.read_csv("crime_data.csv")
crimes["Rate"]=crimes["Total"]/crimes["Population"]
desc=crimes.describe()
meanCrimeRate=desc["Rate"]["mean"]

print("Mean crime rate:",meanCrimeRate)

print("Years when the crime rate was above the mean:",list(crimes["Year"][crimes["Rate"]>meanCrimeRate]))

crimeTypes=["Violent","Property","Murder","Forcible Rape","Robbery","Aggravated Assault","Burglary","Larceny","Vehicle Theft"]
leastCommonCrime="Violent"
leastCommonMean=desc["Violent"]["mean"]
for i in range(len(crimeTypes)):
    if desc[crimeTypes[i]]["mean"]<leastCommonMean:
        leastCommonCrime=crimeTypes[i]
        leastCommonMean=desc[crimeTypes[i]]["mean"]
print("Least common crime:",leastCommonCrime,"(Mean:",str(leastCommonMean)+")")

closeMeans1="Violent"
closeMeans2="Property"
closeMean=abs(desc["Violent"]["mean"]-desc["Property"]["mean"])
for i in range(len(crimeTypes)-1):
    crime1=crimeTypes[i]
    for j in range(i+1,len(crimeTypes)):
        crime2=crimeTypes[j]
        if abs(desc[crime1]["mean"]-desc[crime2]["mean"])<closeMean:
            closeMeans1=crime1
            closeMeans2=crime2
            closeMean=abs(desc[crime1]["mean"]-desc[crime2]["mean"])
            
print("Two crimes with close means:",closeMeans1+",",closeMeans2)
#describe correlation between the two

#desc[[closeMeans1,closeMeans2]]["mean"]
print(crimes[[closeMeans1,closeMeans2]].corr())
print("Muder and forcible rape have a positive correlation of approximately 0.69, which is moderately high.")

Mean crime rate: 0.04236939729496593
Years when the crime rate was above the mean: [1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]
Least common crime: Murder (Mean: 17289.40350877193)
Two crimes with close means: Murder, Forcible Rape
                 Murder  Forcible Rape
Murder         1.000000       0.687251
Forcible Rape  0.687251       1.000000
Muder and forcible rape have a positive correlation of approximately 0.69, which is moderately high.


## Problem 2
Answer the following questions using the pandas numerical methods. Support your answers with data from the file.

1) In what year did each crime hit an all-time high in the U.S.? In what year did the majority of crimes hit an all-time high?

2) Which decade had the greatest amount of total crime?

3) During this decade, which crime was commited most, and what percent of the total crime was it?

In [29]:
maxCrimeYears=[]
maxCrimeCount=[]
for i in range(len(crimeTypes)):
    maxCrimeYears.append(0)
    maxCrimeCount.append(0)
for i in range(len(crimeTypes)):
    for j in range(len(crimes["Year"])):
        if crimes[crimeTypes[i]][j]>maxCrimeCount[i]:
            maxCrimeYears[i]=crimes["Year"][j]
            maxCrimeCount[i]=crimes[crimeTypes[i]][j]
for i in range(len(crimeTypes)):
    print(crimeTypes[i],maxCrimeYears[i],maxCrimeCount[i])
print("Majority of the crimes hit an all-time high in 1991\n")

decade_start = 1960
decade_end = 1969
decade_crimes = 0

for i in range(1960,1970):
    decade_crimes+=int(crimes[crimes["Year"] == i][["Total"]].sum())

for i in range(1961,2008):
    curr_crimes = decade_crimes
    curr_crimes -= int(crimes[crimes["Year"] == i-1][["Total"]].sum())
    curr_crimes += int(crimes[crimes["Year"] == i+9][["Total"]].sum())
    if curr_crimes > decade_crimes:
        decade_start = i
        decade_end = i+9
        decade_crimes = curr_crimes
    
print("The decade with the largest crime was from",decade_start,"to",decade_end,"(",int(decade_crimes),")\n")

print("Crimes from 1987-1996:")
violent_decade = crimes[(decade_start <= crimes["Year"]) & (crimes["Year"] <= decade_end)][crimeTypes]
for i in range(len(crimeTypes)):
    print(crimeTypes[i],int(violent_decade[crimeTypes[i]].sum()))
    
property_rate = round(100*int(violent_decade["Property"].sum())/decade_crimes,2)
print("During this decade, Property crimes were commited the most, making "+str(property_rate)+"% of the crime rate")

Violent 1992 1932270
Property 1991 12961100
Murder 1991 24700
Forcible Rape 1992 109060
Robbery 1991 687730
Aggravated Assault 1993 1135610
Burglary 1980 3795200
Larceny 1991 8142200
Vehicle Theft 1991 1661700
Majority of the crimes hit an all-time high in 1991

The decade with the largest crime was from 1987 to 1996 ( 140960763 )

Crimes from 1987-1996:
Violent 17631449
Property 123329400
Murder 223296
Forcible Rape 998260
Robbery 6033404
Aggravated Assault 10376508
Burglary 29481284
Larceny 78684400
Vehicle Theft 15163774
During this decade, Property crimes were commited the most, making 87.49% of the crime rate


## Problem 3
Using panda commands, find the crime rates of the years where Property crimes were less than 90% of the total crime and burglary crimes were more than 20% of the total crime or property crimes were less than 90% of the total crime while violent crimes were more than 12% of the total crime.

In [86]:
#SELECT Year, Rate FROM crimes WHERE Property < 0.9*Total AND (Burglary > 0.2*Total OR Violent > 0.12*Total)
crimes[(crimes["Property"] < 0.9*crimes["Total"]) & ((crimes["Burglary"] > 0.2*crimes["Total"]) | (crimes["Violent"] > 0.12*crimes["Total"]))][["Year","Total","Violent","Property","Burglary","Rate"]]

,Year,Total,Violent,Property,Burglary,Rate
12,1972,8248800,834900,7413900,2375500,0.039614
13,1973,8718100,875910,7842200,2565500,0.041544
20,1980,13408300,1344520,12063700,3795200,0.059500
21,1981,13423800,1361820,12061900,3779700,0.058582
22,1982,12974400,1322390,11652000,3447100,0.056037
23,1983,12108600,1258090,10850500,3129900,0.051750
24,1984,11881800,1273280,10608500,2984400,0.050313
25,1985,12431400,1328800,11102600,3073300,0.052071
26,1986,13211869,1489169,11722700,3241410,0.055019
27,1987,13508700,1483999,12024700,3236184,0.055756


## Problem 4
Read in the files *final_accidents2.pickle* and *final_drivers.pickle* using the function *read_pickle()*.  
The accidents file contains data on various accidents that occurred in the U.S. and their causes.
The drivers data has data on various tickets given to drivers, how many tickets given, and what the ticket is for.

Create a dataframe containing case number, age, and fatalities using an inner join operation on the Case Number. In this dataframe, also include the speed column from each dataframe.
Create a second dataframe using an outer join.

Use these dataframe to answer the following questions. Make sure to support your claims with appropriate and presentable data.

1. What is the most common kind of weather involved with accidents?

2. How many accidents involved speeding?

3. What is the most common type of speeding in accidents?

4. Rounding to the nearest integer, what is the mean number of drunk drivers involved in each accident?

5. What is the mean number of fatalities per accident each year?

6. What month has the most fatalities each year?

In [27]:
accidents=pd.read_pickle("final_accidents2.pickle")
drivers=pd.read_pickle("final_drivers.pickle")

df1=pd.merge(accidents,drivers,on="ST_CASE")[["ST_CASE","AGE","FATALS","SP","SPEEDREL"]]
df2=pd.merge(accidents,drivers,on=["ST_CASE","YEAR"],how="outer").dropna()

weather=[0,1,2,3,4,5,6,7,8,9,10,11,12,98,99]
fatal_weather=0
weather_total=0

for i in range(len(weather)):
    if len(df2[df2["WEATHER"]==weather[i]]) > weather_total:
        fatal_weather=weather[i]
        weather_total=len(df2[df2["WEATHER"]==weather[i]])
print("Most common kind of weather involved in accidents:",fatal_weather,"(Clear)")

print("Accidents that involved speeding:",len(df1[df1["SP"]==1]))

fatal_speed=0
speed_total=0
for i in range(10):
    if len(df2[df2["SPEEDREL"]==i]) > speed_total:
        fatal_speed=i
        speed_total=len(df2[df2["SPEEDREL"]==i])
print("Most common type of speeding in accidents:",fatal_speed,"(No speeding)")

print("Average number of drunk drivers involved in each accident:",round(df2["DRUNK_DR"].mean()))
print("Average fatalities per accident (2010):",df2[df2["YEAR"]==2010]["FATALS"].mean())
print("Average fatalities per accident (2011):",df2[df2["YEAR"]==2011]["FATALS"].mean())
print("Average fatalities per accident (2012):",df2[df2["YEAR"]==2012]["FATALS"].mean())
print("Average fatalities per accident (2013):",df2[df2["YEAR"]==2013]["FATALS"].mean())
print("Average fatalities per accident (2014):",df2[df2["YEAR"]==2014]["FATALS"].mean())

fatal_month=1
month_total=df2[df2["MONTH"]==fatal_month]["FATALS"].sum()
months=["January","February","March","April","May","June","July","August","September","October","November","December"]
for i in range(12):
    if df2[df2["MONTH"]==i+1]["FATALS"].sum() > month_total:
        fatal_month=i+1
        month_total=df2[df2["MONTH"]==i+1]["FATALS"].sum()
print("Month with most fatalities each year:",months[fatal_month-1])

Most common kind of weather involved in accidents: 1 (Clear)
Accidents that involved speeding: 309343
Most common type of speeding in accidents: 0 (No speeding)
Average number of drunk drivers involved in each accident: 0
Average fatalities per accident (2010): 1.11195086309252
Average fatalities per accident (2011): 1.1040130278003955
Average fatalities per accident (2012): 1.1099647934399928
Average fatalities per accident (2013): 1.1093458793581525
Average fatalities per accident (2014): 1.1151581292772208
Month with most fatalities each year: August
